Full code to make all waypoints from road sections 

In [1]:
#all imports
import carla #the sim library itself
import time # to set a delay after each photo
import numpy as np #in this example to change image representation - re-shaping
import math
import sys
import random
import glob
import os
try:
    sys.path.append(glob.glob('./carla/dist/carla-0.9.14-py3.7-win-amd64.egg')[0])
except IndexError:
    print('Couldn\'t import Carla egg properly')
import carla

try:
    sys.path.insert(0,r'C:\Users\ashokkumar\source\repos\AD\carla')
except IndexError:
    pass

from carla import ColorConverter as cc
from agents.navigation.global_route_planner import GlobalRoutePlanner
from scipy.spatial import KDTree

SECONDS_PER_EPISODE = 25

SECONDS_PER_EPISODE = 10  # seconds per episode maybe needs to be removed later
FIXED_DELTA_SECONDS = 0.02 # seconds per frame
SHOW_PREVIEW = True # Show preview for debugging
NO_RENDERING = False # No rendering for training
SYNCRONOUST_MODE = False # Synchronous mode for training
SPIN = 10   # angle of spin for initail spawn
HEIGHT = 480 # height of image
WIDTH = 640 # width of image
SHOW_CAM = SHOW_PREVIEW     # render camera
front_camera = None
CAMERA_POS_Z = 1.3      # camera position similar to tesla model 3
CAMERA_POS_X = 1.4    # camera position similar to tesla model 3
im_width = WIDTH
im_height = HEIGHT


In [2]:
## INITIALIZE THE SIMULATOR

# connect to the sim 
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()
world = client.load_world('Town02')

settings = world.get_settings()
settings.no_rendering_mode = NO_RENDERING
settings.synchronous_mode = SYNCRONOUST_MODE
settings.fixed_delta_seconds = FIXED_DELTA_SECONDS
world.apply_settings(settings)
blueprint_library = world.get_blueprint_library()
actor_list = []
vehicle = None
collision_hist = []

# get map look at the map
if SHOW_CAM:
    spectator = world.get_spectator()

town_map = world.get_map()
roads = town_map.get_topology()



In [3]:
# Define the waypoint and KDTree generation logic

town_map = world.get_map()
waypoints = town_map.generate_waypoints(4)
waypoint_positions = np.array([(wp.transform.location.x, wp.transform.location.y) for wp in waypoints])
waypoint_tree = KDTree(waypoint_positions)
spawn_position = None        
start_waypoint = None
dest_waypoint = None
observation = None

print(f"Generated {len(waypoints)} waypoints")
# draw all point in the sim for 60 seconds
for wp in waypoints:
    world.debug.draw_string(wp.transform.location, '^', draw_shadow=False,
        color=carla.Color(r=0, g=0, b=255), life_time=20.0,
        persistent_lines=True)



Generated 770 waypoints


### Functions

In [4]:
## CLEANUP THE SIMULATOR
def cleanup():
    for actor in actor_list:
        actor.destroy()
        actor_list.pop(actor_list.index(actor))                  
    world.tick()


In [5]:
## SPAWN THE VEHICLE
def spawn_vehicle(spawn_point):
    transform = spawn_point
    blueprint_library = world.get_blueprint_library()
    model_3 = blueprint_library.find('vehicle.tesla.model3')
    vehicle = None
    while vehicle is None:
        try:
            vehicle = world.spawn_actor(model_3, transform)
        except Exception as e:
            print(f"Failed to spawn vehicle: {e}")
            time.sleep(0.2)  # Brief delay before retrying
    return vehicle

In [6]:
def initial_waypoint(initial_pos):
    # Query the KD-tree for the closest waypoint to initial_pos
    _, closest_index = waypoint_tree.query([initial_pos.x, initial_pos.y])
    closest_waypoint = waypoints[closest_index]
    return closest_waypoint

def next_waypoint(start_waypoint, look_ahead=5.0):
    next_wps = start_waypoint.next(look_ahead)
    if next_wps:
        return next_wps[0]
    else:
        # Handle the case where no next waypoint is found
        print("No next waypoint found")
        return None
    
def draw_waypoint(waypoint, color=carla.Color(255, 0, 0), lt=20.0):
    world.debug.draw_string(waypoint.transform.location, 'O', draw_shadow=False,
        color=color, life_time=lt, persistent_lines=True)
    
def vector3d_to_location(vector):
    return carla.Location(x=vector.x, y=vector.y, z=vector.z)

def draw_arrow(start_location, end_location,color=carla.Color(255, 0, 0), lt=20.0):
    if start_location and end_location:
        start_location = vector3d_to_location(start_location)
        end_location = vector3d_to_location(end_location)
        end_location += carla.Location(z=2)
        start_location += carla.Location(z=2)
        world.debug.draw_arrow(start_location, end_location, thickness=0.1, arrow_size=0.3, color=color, life_time=lt)

def draw_heading_arrow(location, yaw, length=5.0, color=carla.Color(255, 0, 0), life_time=5.0):
    yaw_corrected = (yaw) % 360

    # Convert yaw from degrees to radians
    yaw_rad = np.deg2rad(yaw_corrected)

    # Compute the direction vector based on the corrected yaw (heading)
    direction_x = np.cos(yaw_rad)
    direction_y = np.sin(yaw_rad)

    # Calculate the end location of the arrow using the direction vector
    end_location = carla.Location(
        x=location.x + length * direction_x,
        y=location.y + length * direction_y,
        z=2
    )
    location.z = 2
    # Draw the arrow in the CARLA world
    world.debug.draw_arrow(
        location,                 # Start point (the given location)
        end_location,             # End point (direction of the yaw)
        thickness=0.1,            # Arrow thickness
        arrow_size=0.3,           # Arrowhead size
        color=color,              # Color of the arrow
        life_time=life_time       # How long the arrow will stay in the world (in seconds)
    )

def draw_relative_heading_arrow(vehicle_transform, waypoint_transform, length=5.0, color=carla.Color(0, 255, 0), life_time=5.0):
# Extract vehicle and waypoint yaw in degrees
    vehicle_yaw = vehicle_transform.rotation.yaw
    waypoint_yaw = waypoint_transform.rotation.yaw

    # Calculate the relative heading (difference between vehicle and waypoint yaw)
    relative_heading = (vehicle_yaw - waypoint_yaw) % 360
    if relative_heading > 180:
        relative_heading -= 360

    # The correct heading to draw the arrow is the vehicle's global yaw minus the relative heading
    absolute_heading = vehicle_yaw - relative_heading

    # Convert the absolute heading to radians
    absolute_heading_rad = np.deg2rad(absolute_heading)

    # Get the vehicle's current location
    vehicle_location = vehicle_transform.location

    # Compute the arrow's end location based on the absolute heading direction
    arrow_end_location = carla.Location(
        x=vehicle_location.x + length * np.cos(absolute_heading_rad),
        y=vehicle_location.y + length * np.sin(absolute_heading_rad),
        z=vehicle_location.z
    )

    # Draw an arrow in the CARLA world from the vehicle location in the direction of absolute heading
    world.debug.draw_arrow(
        vehicle_location,             # Start point (vehicle's current location)
        arrow_end_location,           # End point (direction of the relative heading)
        thickness=0.1,                # Arrow thickness
        arrow_size=0.3,               # Arrowhead size
        color=color,                  # Color of the arrow
        life_time=life_time           # How long the arrow will stay in the world (in seconds)
    )




In [7]:
## OBSERVATION LOGIC - 1
def get_speed(v, max_speed=50.0):
    if not isinstance(v, carla.Vector3D):
        print("Invalid velocity vector provided.")
        return 0
    speed = int(3.6 * np.sqrt(v.x**2 + v.y**2 + v.z**2))
    normalized_speed = np.clip(speed / max_speed, -1, 1)  # Normalize speed to [-1, 1]
    return normalized_speed

def get_relative_heading(vehicle_transform, waypoint):
    # Extract vehicle and waypoint yaw in degrees
    vehicle_yaw = vehicle_transform.rotation.yaw
    current_waypoint = waypoint  # Start from the current waypoint
    num_lookahead = 5  # Number of waypoints to look ahead
    distance_lookahead = 3.0
    yaw_differences_sum = 0  # Accumulator for the sum of yaw differences
    turn_direction = turn_directon(vehicle_transform, waypoint, num_lookahead, distance_lookahead)
    
    for i in range(1, num_lookahead + 1):
        # Fetch the next waypoint 'i' steps ahead
        future_waypoint = next_waypoint(current_waypoint, distance_lookahead)
        
        if not future_waypoint:
            break  # Stop if there's no more valid waypoints
    
        # Check if the current waypoint is part of a turn
        future_yaw = future_waypoint.transform.rotation.yaw
        yaw_diff = abs(current_waypoint.transform.rotation.yaw - future_yaw) % 360
        yaw_diff = yaw_diff if yaw_diff <= 180 else 360 - yaw_diff
        
        # Accumulate the yaw difference
        yaw_differences_sum += yaw_diff
        
        # Move to the next waypoint for the next iteration
        current_waypoint = future_waypoint

    vehicle_yaw = vehicle_yaw % 360            
    # Calculate relative heading as the difference between the yaw sum and the vehicle's yaw
    relative_heading = (yaw_differences_sum - vehicle_yaw) % 360
    
    # Normalize to [-180, 180] range
    if relative_heading > 180:
        relative_heading -= 360

    # Adjust the heading based on the turn direction
    normalized_heading = turn_direction * (relative_heading / 180.0)
    return np.clip(normalized_heading, -1, 1)

def turn_directon(vehicle_transform, waypoint, num_lookahead=5, distance_lookahead=3.0):
    # Extract yaw angles from waypoints
    vehicle_yaw = vehicle_transform.rotation.yaw  # Vehicle's current yaw
    current_waypoint = waypoint  # Start from the current waypoint
    future_waypoint = None
    for i in range(1, num_lookahead + 1):
        future_waypoint = next_waypoint(current_waypoint, distance_lookahead)
        if not future_waypoint:
            break  # Stop if there's no valid future waypoint
        current_waypoint = future_waypoint  # Move to the next waypoint

    if future_waypoint:
        # Get the yaw of the future waypoint
        future_yaw = future_waypoint.transform.rotation.yaw
        
        # Calculate the yaw difference between the vehicle and the future waypoint
        yaw_diff = (future_yaw - vehicle_yaw) % 360
        
        # Normalize the yaw difference to the [-180, 180] range
        if yaw_diff > 180:
            yaw_diff -= 360
        if yaw_diff > 0:  # If the future yaw is greater than the vehicle's yaw by a threshold
            return 1
        else:
            return -1

In [8]:
   
def get_lateral_distance(vehicle_transform, waypoint, max_distance=3.0):
    # Get the vehicle and waypoint positions
    waypoint_location = waypoint.transform.location
    waypoint_forward_vector = waypoint.transform.get_forward_vector()
    vehicle_location = vehicle_transform.location

    # Convert CARLA location objects to NumPy arrays for easier vector calculations
    waypoint_location_np = np.array([waypoint_location.x, waypoint_location.y])
    waypoint_forward_np = np.array([waypoint_forward_vector.x, waypoint_forward_vector.y])
    vehicle_location_np = np.array([vehicle_location.x, vehicle_location.y])

    # Normalize the waypoint's forward vector (this is the direction of the lane)
    norm = np.linalg.norm(waypoint_forward_np)
    if norm == 0:
        return 0  # Prevent division by zero if the forward vector length is zero
    waypoint_forward_np = waypoint_forward_np / norm

    # Vector from the waypoint (center of the lane) to the vehicle
    vehicle_vector_np = vehicle_location_np - waypoint_location_np

    # Project the vehicle vector onto the waypoint's forward vector (lane direction)
    projection_length = np.dot(vehicle_vector_np, waypoint_forward_np)
    projection_point = projection_length * waypoint_forward_np

    # Calculate the lateral distance between the vehicle and the center line
    lateral_vector = vehicle_vector_np - projection_point
    lateral_distance = np.linalg.norm(lateral_vector)

    # Normalize the lateral distance to the range [-1, 1]
    normalized_lateral_distance = np.clip(lateral_distance / max_distance, -1, 1)

    # Check if the vehicle is to the left or right of the lane center by using a cross product
    cross_product = np.cross(waypoint_forward_np, vehicle_vector_np)
    if cross_product < 0:
        # If the cross product is negative, the vehicle is to the right of the lane
        normalized_lateral_distance *= -1

    return normalized_lateral_distance

In [9]:
## OBSERVATION LOGIC - 2

def get_observation(vehicle_transform, dest_waypoint):
        if vehicle and vehicle.is_alive:
                velocity = vehicle.get_velocity()
                speed = get_speed(velocity)
        else:
                speed = 0  # Default speed if vehicle is not available
        lateral_distance = get_lateral_distance(vehicle_transform, dest_waypoint)
        heading = get_relative_heading(vehicle_transform, dest_waypoint)
        
        print(f"lateral_distance: {lateral_distance}, speed: {speed}, heading: {heading}")
        obs = np.array([
                lateral_distance,
                speed,
                heading,
        ], dtype=np.float32)
        return obs

In [10]:
spawn_points = world.get_map().get_spawn_points()

for i, spawn_point in enumerate(spawn_points):
    # Draw in the spectator window the spawn point index
    world.debug.draw_string(spawn_point.location, str(i), life_time=400)
    # We can also draw an arrow to see the orientation of the spawn point
    # (i.e. which way the vehicle will be facing when spawned)
    world.debug.draw_arrow(spawn_point.location, spawn_point.location + spawn_point.get_forward_vector(), life_time=100)



In [11]:
print(f"{spawn_points[0].location}")

Location(x=-7.530000, y=142.190002, z=0.500000)


### Understanding Spawn points and Orientation

In [ ]:
print(spawn_points[36])
spawn_position = spawn_points[36]

print(spawn_position)
start_waypoint = initial_waypoint(spawn_position.location)
start_waypoint.transform.location.x += 1
draw_waypoint(start_waypoint, color=carla.Color(0, 255, 0), lt=20.0)
print(start_waypoint)

In [ ]:
## RESET THE SIMULATOR
cleanup()
vehicle = spawn_vehicle(spawn_position)
actor_list.append(vehicle)
world.tick()
vehicle_transform = vehicle.get_transform()
start_waypoint = initial_waypoint(vehicle_transform.location)
dest_waypoint = next_waypoint(start_waypoint, 3)
print("Vehicle spawned at location ", vehicle_transform.location)

spectator.set_transform(carla.Transform(vehicle_transform.location + carla.Location(z=40), carla.Rotation(pitch=-90)))

In [ ]:

print("Start waypoint: ", start_waypoint.transform)
print("Vehicle transform: ", vehicle_transform)
print ("dest_waypoint: ", dest_waypoint)
draw_waypoint(dest_waypoint, color=carla.Color(0, 255, 0), lt=20.0)


In [ ]:

vehicle.set_transform(vehicle_transform)
print("Vehicle transform after rotation: ", vehicle_transform)

In [ ]:
print("Relative heading: ", get_relative_heading(vehicle_transform, dest_waypoint))

In [ ]:
vehicle.apply_control(carla.VehicleControl(throttle=0.0, steer=0.0, brake=1.0))
speed = get_speed(vehicle.get_velocity())
print (vehicle.get_velocity())
print("Speed: ", speed)
vehicle_transform = vehicle.get_transform()
start_waypoint = initial_waypoint(vehicle_transform.location)
vehicle.set_transform(start_waypoint.transform)
world.tick()
print("Vehicle moved to location ", vehicle.get_transform().location)
vehicle_transform = vehicle.get_transform()
print(vehicle_transform)

In [ ]:
world.wait_for_tick()
dest_waypoint = next_waypoint(start_waypoint, look_ahead=3.0)
print("Start waypoint: ", start_waypoint.transform)    
print("Dest waypoint: ", dest_waypoint.transform)
print(vehicle_transform)
draw_waypoint(dest_waypoint, carla.Color(255, 0, 0))
draw_waypoint(start_waypoint, carla.Color(0, 255, 0))

In [ ]:
print(get_observation(vehicle_transform, dest_waypoint))


### Understanding Lateral Distance

In [ ]:
## Geting inputs and coverting to numpy arrays
waypoint_location = dest_waypoint.transform.location
waypoint_forward_vector = dest_waypoint.transform.get_forward_vector()
vehicle_location = vehicle_transform.location
print(waypoint_location, waypoint_forward_vector, vehicle_location)
# Convert CARLA location objects to NumPy arrays for easier vector calculations
waypoint_location_np = np.array([waypoint_location.x, waypoint_location.y])
waypoint_forward_np = np.array([waypoint_forward_vector.x, waypoint_forward_vector.y])
vehicle_location_np = np.array([vehicle_location.x, vehicle_location.y])
print(waypoint_location_np, waypoint_forward_np, vehicle_location_np)
# Normalize the waypoint's forward vector (this is the direction of the lane)
norm = np.linalg.norm(waypoint_forward_np)
print(norm)
waypoint_forward_np = waypoint_forward_np / norm
print(waypoint_forward_np)


In [ ]:
## 
# Vector from the waypoint (center of the lane) to the vehicle
vehicle_vector_np = vehicle_location_np - waypoint_location_np

# Project the vehicle vector onto the waypoint's forward vector (lane direction)
projection_length = np.dot(vehicle_vector_np, waypoint_forward_np)
projection_point = projection_length * waypoint_forward_np

lateral_vector = vehicle_vector_np - projection_point
lateral_distance = np.linalg.norm(lateral_vector)

# Normalize the lateral distance to the range [-1, 1]
normalized_lateral_distance = np.clip(lateral_distance / 3, -1, 1)

# Check if the vehicle is to the left or right of the lane center by using a cross product
cross_product = np.cross(waypoint_forward_np, vehicle_vector_np)
if cross_product < 0:
    # If the cross product is negative, the vehicle is to the right of the lane
    normalized_lateral_distance *= -1
print(normalized_lateral_distance)


### Understanding Relative Heading 

In [ ]:
print(spawn_points[36])
spawn_position = spawn_points[36]
print(spawn_position)
start_waypoint = initial_waypoint(spawn_position.location)
start_waypoint.transform.location.x += 1
draw_waypoint(start_waypoint, color=carla.Color(0, 255, 0), lt=20.0)
print(start_waypoint)

In [ ]:
## RESET THE SIMULATOR
cleanup()
vehicle = spawn_vehicle(spawn_position) # Use 9,36,43
actor_list.append(vehicle)
world.tick()
vehicle_transform = vehicle.get_transform()
start_waypoint = initial_waypoint(vehicle_transform.location)
world.tick()
start_waypoint = initial_waypoint(vehicle_transform.location)
dest_waypoint = next_waypoint(start_waypoint, 3)
print("Vehicle spawned at location ", vehicle_transform.location)

spectator.set_transform(carla.Transform(vehicle_transform.location + carla.Location(z=40), carla.Rotation(pitch=-90)))

In [ ]:
print("Start waypoint: ", start_waypoint.transform)
print("Vehicle transform: ", vehicle_transform)
print ("dest_waypoint: ", dest_waypoint.transform)

In [ ]:
# Extract vehicle and waypoint yaw in degrees
#vehicle_transform.rotation.yaw = 190
vehicle.set_transform(vehicle_transform)
waypoint = start_waypoint # Start from the current waypoint
vehicle_yaw = vehicle_transform.rotation.yaw
current_waypoint = start_waypoint
waypoint_yaw = waypoint.transform.rotation.yaw
num_lookahead = 5  # Number of waypoints to look ahead
distance_lookahead = 3.0
yaw_differences_sum = 0  # Accumulator for the sum of yaw differences
print(vehicle_yaw, waypoint_yaw)
turn = turn_directon(vehicle_transform, waypoint, num_lookahead, distance_lookahead)
if turn == -1:
    print("Turning left")
else:
    print("Turning right")

In [ ]:
def normalize_angle(angle):
    """Normalize an angle to the range [-180, 180]."""
    angle = angle % 360  # First, wrap the angle to [0, 360]
    if angle > 180:
        angle -= 360  # Bring it to [-180, 180]
    return angle
print(normalize_angle(vehicle_yaw), normalize_angle(waypoint_yaw))


In [ ]:

for i in range(1, num_lookahead + 1):
    # Fetch the next waypoint 'i' steps ahead
    future_waypoint = next_waypoint(current_waypoint, distance_lookahead)
    if not future_waypoint:
        break  # Stop if there's no more valid waypoints

    # Check if the current waypoint is part of a turn
    future_yaw = future_waypoint.transform.rotation.yaw
    yaw_diff = abs(current_waypoint.transform.rotation.yaw - future_yaw) % 360
    yaw_diff = yaw_diff if yaw_diff <= 180 else 360 - yaw_diff
    
    # Accumulate the yaw difference
    yaw_differences_sum += yaw_diff
    print(f"future_yaw: {future_yaw}, yaw_diff: {yaw_diff}, yaw_differences_sum: {yaw_differences_sum}")
    # Move to the next waypoint for the next iteration
    current_waypoint = future_waypoint
    draw_waypoint(future_waypoint, carla.Color(0, 255, 0))

vehicle_yaw = vehicle_yaw % 360     
relative_yaw =(vehicle_yaw - waypoint_yaw) % 360
print(relative_yaw)       
# Calculate relative heading as the difference between the yaw sum and the vehicle's yaw
relative_heading = (yaw_differences_sum - turn*relative_yaw) % 360
print(relative_heading)
# Normalize to [-180, 180] range
if relative_heading > 180:
    relative_heading -= 360

# Adjust the heading based on the turn direction
normalized_heading = turn * (relative_heading / 180.0)
print( np.clip(normalized_heading, -1, 1))


In [ ]:

def turn_directon(vehicle_transform, waypoint, num_lookahead=5, distance_lookahead=3.0):
    # Extract yaw angles from waypoints
    vehicle_yaw = vehicle_transform.rotation.yaw  # Vehicle's current yaw
    current_waypoint = waypoint  # Start from the current waypoint
    future_waypoint = None
    for i in range(1, num_lookahead + 1):
        future_waypoint = next_waypoint(current_waypoint, distance_lookahead)
        if not future_waypoint:
            break  # Stop if there's no valid future waypoint
        current_waypoint = future_waypoint  # Move to the next waypoint

    if future_waypoint:
        # Get the yaw of the future waypoint
        future_yaw = future_waypoint.transform.rotation.yaw
        
        # Calculate the yaw difference between the vehicle and the future waypoint
        yaw_diff = (future_yaw - vehicle_yaw) % 360
        
        # Normalize the yaw difference to the [-180, 180] range
        if yaw_diff > 180:
            yaw_diff -= 360
        if yaw_diff > 10:  # If the future yaw is greater than the vehicle's yaw by a threshold
            return 1
        else:
            return -1

In [ ]:
print(get_observation(vehicle_transform, dest_waypoint))


In [ ]:

num_lookahead = 5
distance_lookahead = 3.0
turn_direction = 1
# Extract vehicle and waypoint yaw in degrees
vehicle_yaw = vehicle_transform.rotation.yaw
waypoint_yaw = dest_waypoint.transform.rotation.yaw
draw_heading_arrow(vehicle_transform.location, vehicle_transform.rotation.yaw,color=carla.Color(0, 255, 0))
draw_heading_arrow(dest_waypoint.transform.location, dest_waypoint.transform.rotation.yaw)
# List to store yaw values of waypoints ahead
yaw_differences_sum = 0  # Accumulator for the sum of yaw differences

current_waypoint = dest_waypoint  # Start from the current waypoint


In [ ]:

for i in range(1, num_lookahead + 1):
    # Fetch the next waypoint 'i' steps ahead
    future_waypoint = next_waypoint(current_waypoint, distance_lookahead)
    draw_waypoint(future_waypoint, carla.Color(0, 0, 255))
    if not future_waypoint:
        break  # Stop if there's no more valid waypoints
    # Calculate yaw difference between current waypoint and future waypoint
    future_yaw = future_waypoint.transform.rotation.yaw
    draw_heading_arrow(future_waypoint.transform.location, future_yaw)
    yaw_diff = (current_waypoint.transform.rotation.yaw - future_yaw) % 360
    print(yaw_diff)
    if yaw_diff > 180:
        yaw_diff -= 360
    # Accumulate the yaw difference
    yaw_differences_sum += yaw_diff
    # Move to the next waypoint for the next iteration
    current_waypoint = future_waypoint


In [ ]:
print(yaw_differences_sum)
# If we gathered yaw differences, process the sum
if abs(yaw_differences_sum) > 25:
    # Calculate relative heading as the difference between the yaw sum and the vehicle's yaw
    relative_heading = (yaw_differences_sum) % 360
    
    # Normalize to [-180, 180] range
    if relative_heading > 180:
        relative_heading -= 360

    # Adjust the heading based on the turn direction
    normalized_heading = turn_direction * (relative_heading / 180.0)
    np.clip(normalized_heading, -1, 1)
    print(normalized_heading)

In [ ]:

# If no turn detected, calculate the basic relative heading from the closest waypoint
relative_heading = (waypoint_yaw - vehicle_yaw) % 360

# Normalize to [-180, 180] range
if relative_heading > 180:
    relative_heading -= 360

normalized_heading = relative_heading / 180.0
np.clip(normalized_heading, -1, 1)
print(normalized_heading)


### Route Planner


In [13]:
spawn_points = world.get_map().get_spawn_points()

cleanup()

In [14]:
world_map = world.get_map()
spawn_points = world_map.get_spawn_points()
sampling_resolution = 3
route_planner = GlobalRoutePlanner(world.get_map(), sampling_resolution)


In [15]:
def generate_route(start_waypoint=None):

    if start_waypoint is None:
        start_waypoint = random.choice(spawn_points)
        print(f"Start waypoint: {start_waypoint.location}")
    else:
        start_waypoint = start_waypoint
    dest_waypoint = random.choice(spawn_points)
    distance = start_waypoint.location.distance(dest_waypoint.location)
    while distance < 50:
        dest_waypoint = random.choice(spawn_points)
        distance = start_waypoint.location.distance(dest_waypoint.location)
    try:
        route = route_planner.trace_route(start_waypoint.location, dest_waypoint.location)
    except Exception as e:
        print(f"Failed to generate route: {e}")
        return None
    
    return route 


In [16]:
start_waypoint = random.choice(spawn_points)
route = generate_route(start_waypoint)
for waypoint in route:
    world.debug.draw_string(waypoint[0].transform.location, '^', draw_shadow=False,
                            color=carla.Color(r=0, g=0, b=255), life_time=60.0,
                            persistent_lines=True)

In [17]:

vehicle = spawn_vehicle(start_waypoint)
actor_list.append(vehicle)

In [18]:
spectator.set_transform(carla.Transform(vehicle.get_transform().location + carla.Location(z=40), carla.Rotation(pitch=-90)))


In [31]:
#Draw the route in the simulator
for waypoints in route:
    draw_waypoint(waypoints[0], carla.Color(255, 0, 0))

In [ ]:
vehicle_transform = vehicle.get_transform()
print(vehicle_transform)
previous_waypoints = []


Transform(Location(x=26.940020, y=302.570007, z=0.221618), Rotation(pitch=0.000000, yaw=-179.999634, roll=0.000000))


In [22]:

def find_current_waypoint(vehicle_location,route):
    curr_waypoint = None
    min_distance = 3.0
  
    if route is None:
        print('Route not found')
        return None
    for waypoint, roadinfo  in route:
        dist = waypoint.transform.location.distance(vehicle_location)
        if dist < min_distance:
            min_distance = dist
            curr_waypoint = waypoint
    if curr_waypoint is not None:
        print(f"Current waypoint: {curr_waypoint.transform.location}")
    else:
        print("No waypoint found")

    return curr_waypoint


In [34]:
curr_wp = find_current_waypoint(vehicle_transform.location, route)


Current waypoint: Location(x=-3.370002, y=294.696808, z=0.000000)


In [62]:
vehicle_transform = random.choice(route)[0].transform
vehicle_transform.location.z = 0.5
vehicle_transform.location.x += 2

print(vehicle_transform)
vehicle.set_transform(vehicle_transform)
spectator.set_transform(carla.Transform(vehicle_transform.location + carla.Location(z=40), carla.Rotation(pitch=-90)))


Transform(Location(x=125.179070, y=191.608704, z=0.500000), Rotation(pitch=0.000000, yaw=-359.973694, roll=0.000000))


In [63]:
curr_wp = find_current_waypoint(vehicle_transform.location, route)
draw_waypoint(curr_wp, carla.Color(0, 255, 0), lt=20.0)


Current waypoint: Location(x=126.179070, y=191.610077, z=0.000000)


In [64]:
for waypoint in route:
    world.debug.draw_string(waypoint[0].transform.location, '^', draw_shadow=False,
                            color=carla.Color(r=0, g=0, b=255), life_time=60.0,
                            persistent_lines=True)

In [ ]:
## BOTH WORK
def find_next_waypoint(route, current_waypoint, distance_lookahead=6):
    for waypoint, roadinfo in route:
        dist = waypoint.transform.location.distance(current_waypoint.transform.location)
        if dist == 0:
            index = route.index((waypoint, roadinfo))
            if index + 1 < len(route):
                return route[index + distance_lookahead][0]
            else:
                return None
    return None

def find_mod_next_waypoint( current_waypoint, distance_lookahead=6):
    for i, (waypoint, road_option) in enumerate(route):
        dist = waypoint.transform.location.distance(current_waypoint.transform.location)
        if dist == 0:
            if i + 1 < len(route):
                return route[i + distance_lookahead][0]
            else:
                return None
    return None

In [ ]:
next_wp = find_next_waypoint(route, curr_wp, 7)
print(next_wp)
next_wp_1 = find_mod_next_waypoint(curr_wp, 1)
print(next_wp_1)

Waypoint(Transform(Location(x=132.144226, y=205.157974, z=0.000000), Rotation(pitch=360.000000, yaw=90.058083, roll=0.000000)))
Waypoint(Transform(Location(x=128.700058, y=191.716599, z=0.000000), Rotation(pitch=0.000000, yaw=-346.489532, roll=0.000000)))


: 

In [43]:

draw_waypoint(next_wp, carla.Color(0, 255, 0), lt=20.0)
draw_waypoint(curr_wp, carla.Color(255, 0, 0), lt=20.0)

In [ ]:
distance_lookahead = 3.0
for waypoint, roadinfo  in route:
    dist = waypoint.transform.location.distance(curr_wp.transform.location)
    if dist == 0:
        if waypoint + 1 < len(route):
            next_wp = route[i + distance_lookahead][0]



ValueError: <carla.libcarla.Waypoint object at 0x000001E478109040> is not in list

## Test

In [ ]:
wp = next_waypoint(start_waypoint,25)
draw_waypoint(wp, carla.Color(0, 0, 255))

In [ ]:
vehicle.set_transform(wp.transform)
world.wait_for_tick()
vehicle_transform = vehicle.get_transform()
start_waypoint = initial_waypoint(vehicle_transform.location)
spectator.set_transform(carla.Transform(vehicle_transform.location + carla.Location(z=40), carla.Rotation(pitch=-90)))
vehicle.apply_control(carla.VehicleControl(throttle=0.0, steer=0.0, brake=1.0))
print(vehicle_transform)
dest_waypoint = next_waypoint(start_waypoint, look_ahead=3.0)
print("Start waypoint: ", start_waypoint.transform)    
print("Dest waypoint: ", dest_waypoint.transform)
draw_waypoint(dest_waypoint, carla.Color(255, 0, 0))
draw_waypoint(start_waypoint, carla.Color(0, 255, 0))

In [ ]:
print(get_observation(vehicle_transform, dest_waypoint))

In [ ]:
vehicle_transform.location.y -= 5
vehicle.set_transform(vehicle_transform)


In [ ]:
print(get_observation(vehicle_transform, dest_waypoint))